## Explore NOAA L2b Wildfire Product (ABI-L2-FDCF)

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os, sys

sys.path.append('../wildfire/')
from wildfire import wildfire
from wildfire.goes import utilities, scan

### Read L2 File

In [ ]:
year = 2019
dayofyear = 260
hour = 19

l2_path = f'/nobackupp10/tvandal/data/goes16/ABI-L2-FDCF/{year}/{dayofyear}/{hour}/'

l2_file_path = os.path.join(l2_path, os.listdir(l2_path)[0])

ds = xr.open_dataset(l2_file_path)
print(ds.attrs['summary'])
print(f"Number of fires: {ds['total_number_of_pixels_with_fires_detected'].values}")

In [ ]:
ds

In [ ]:
def extract_patches_2d(arr, height, width, stride):
    assert len(arr.shape) >= 2
    H, W = arr.shape[:2]
    ih = np.arange(0,H,stride)
    iw = np.arange(0,W,stride)
    patches = []
    for i in ih:
        i = min(i, H-height)
        for j in iw:
            j = min(j, W-width)
            patches.append(arr[i:i+height, j:j+height][np.newaxis])
    return np.concatenate(patches)

### Histogram of Fire Temperatures and Random Wildfire Plot

In [ ]:
vals = ds.Temp.values
vmin = np.nanmin(vals)
vmax = np.nanmax(vals)
vals_flat = vals.flatten()
vals_flat = vals_flat[np.isfinite(vals_flat)]
print(np.histogram(vals_flat))

patches = extract_patches_2d(vals, 16, 16, 16)
fire_patches = patches[np.any(np.isfinite(patches), axis=(1,2))]
print(fire_patches.shape)
i = np.random.randint(0, len(fire_patches))
plt.imshow(fire_patches[i])

### Read in L1b Radiances

In [ ]:
# load scan and get wildfire map
#
l1_dir = f'/nex/datapool/geonex/public/GOES16/NOAA-L1B/ABI-L1b-RadF/{year}/{dayofyear}/{hour}/'
files = [os.path.join(l1_dir, f) for f in sorted(os.listdir(l1_dir))]
filepaths = utilities.group_filepaths_into_scans(files)
goes_scan = scan.read_netcdfs(local_filepaths=filepaths[0])

In [ ]:
goes_scan.plot(bands=[1,2,3])

In [ ]:
goes_scan_2km = goes_scan.rescale_to_500m() # this method is misnamed in scan.py
datasets = []
band16 = goes_scan_2km['band_16']
for band, s in goes_scan_2km.iteritems():
    d = s.dataset
    d = d.assign_coords(x=band16.dataset.x.values,
                        y=band16.dataset.y.values)
    datasets.append(d['Rad'])
    
l1ds = xr.concat(datasets, 'band')

### Join L1b and L2 Fire Data and Extract Patches

In [ ]:
alldata = np.concatenate([l1ds.values, ds.Temp.values[np.newaxis]], 0)
alldata = np.transpose(alldata, (1,2,0))
alldata_patches = extract_patches_2d(alldata, 16, 16, 16)
print(alldata_patches.shape)

fire_idxs = np.any(np.isfinite(alldata_patches[:,:,:,-1]), axis=(1,2))
alldata_fire_patches = alldata_patches[fire_idxs]
alldata_fire_patches.shape

### Show RGB, Band 15 Temp, and L2 Fire Temp

In [ ]:
#i = np.random.randint(0, len(alldata_fire_patches))
i = 212

fig, axs = plt.subplots(1,3,figsize=(12,5))
axs = np.ravel(axs)
#[a.axis('off') for a in axs]

axs[2].imshow(alldata_fire_patches[i,:,:,-1])
axs[2].set_title("NOAA L2b Fire Temperature")

rgb = alldata_fire_patches[i,:,:,[1,2,0]]
rgb = np.transpose(rgb, (1,2,0))
mn = np.nanmin(rgb)
mx = np.nanmax(rgb)
rgb = (rgb - mn) / (mx - mn)
axs[0].imshow(rgb)
axs[0].set_title("False Color RGB")

axs[1].imshow(alldata_fire_patches[i,:,:,14])
axs[1].set_title("Band 15 - Brightness Temp")